In [ ]:
import pystiche
from pystiche import demo, loss, enc, optim
from pystiche.image import show_image, write_image
from pystiche.misc import get_device, get_input_image

In [ ]:
def get_style_op(encoder, layer_weight):
    return loss.GramLoss(encoder, score_weight=layer_weight)

def bilde_merging(content_bilde, style_bilde, content_weight, style_weight, num_steps, starting_point):
    multi_layer_encoder = enc.vgg19_multi_layer_encoder()
    
    #Innholds-bilde
    content_layer = "relu4_2" 
    content_encoder = multi_layer_encoder.extract_encoder(content_layer)
    content_weight = content_weight
    content_loss = loss.FeatureReconstructionLoss(
                 content_encoder, score_weight=content_weight)
    
    
    #stil-bilde
    style_layers =("relu1_1","relu2_1","relu3_1","relu4_1","relu5_1")
    style_weight = style_weight
    style_loss = loss.MultiLayerEncodingLoss(
                multi_layer_encoder, style_layers,
                get_style_op, score_weight=style_weight,)
    
    
    #samensetter de to loss-funskjonene
    perceptual_loss = loss.PerceptualLoss(content_loss, style_loss).to(device)
    
    
    content_image = content_bilde
    style_image = style_bilde
    
    perceptual_loss.set_content_image(content_image)
    perceptual_loss.set_style_image(style_image)
    #starting_point: "random" = white noise, "content" = content image 
    input_image = get_input_image(starting_point, content_image=content_image)
    show_image(input_image, title="Input image")
    show_image(style_image, title="Style image")
    
    #optimasering
    output_image = optim.image_optimization(input_image, perceptual_loss, num_steps=num_steps)
    
    merged_bilde = output_image
    return merged_bilde

In [ ]:
from pystiche.image import read_image
size = 500
device = get_device()


# prøver frem med ulike bilder fra demosamlingen
images = demo.images()
content = images["bird1"].read(size=size, device=device)
#style = images["mosaic"].read(size=size, device=device)

style = read_image("../input/syrebilde/Trip_1.jpeg", size=size, device=device)
# prøver med ulike stillag når startbildet settes til støy og vekten på innhold til null: 
out_put = bilde_merging(content, style, 0, 1e3, 500, "random")
show_image(out_put, title="Output image")

In [ ]:
# Har prøvd ut mange ulike kombinasjoner og forhold som jeg ikke har med her. mye utforsking. 

In [ ]:
# med halvparten av iterasjonene vi pleier: 
out_put = bilde_merging(content_image, style_image, 1e0, 1e3, 250, "content")
show_image(out_put, title="Output image")

In [ ]:
# med dobbelt så mange itterasjoner som vi pleier: 
out_put = bilde_merging(content_image, style_image, 1e0, 1e3, 1000, "content")
show_image(out_put, title="Output image")

In [ ]:
# lavere vekt, som viser en forskjell 
out_put = bilde_merging(content_image, style_image, 1e0, 1e0, 500, "content")
show_image(out_put, title="Output image")
